In [6]:
from docx import Document
import json
import os

input_folder = "C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data"
output_folder = "C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/json_docs"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(".docx"):
        doc_path = os.path.join(input_folder, filename)
        doc = Document(doc_path)
        result = []
        actual = {}
        for para in doc.paragraphs:
            estilo = para.style.name.lower()
            texto = para.text.strip()
            if not texto:
                continue
            if estilo in ["heading 1", "encabezado 1"]:
                if actual:
                    # Limpia el campo "text" antes de guardar el bloque
                    if "text" in actual:
                        actual["text"] = actual["text"].lstrip()
                    result.append(actual)
                    actual = {}
                actual["title"] = texto
                actual["text"] = ""
            elif estilo == "normal":
                if "text" not in actual:
                    actual["text"] = texto
                else:
                    actual["text"] += "\n" + texto
        if actual:
            if "text" in actual:
                actual["text"] = actual["text"].lstrip()
            result.append(actual)

        # Extrae ley_nombre como antes
        ley_nombre = ""
        for bloque in result:
            if "title" in bloque:
                ley_nombre = bloque["title"]
                break
            elif "text" in bloque and not ley_nombre:
                ley_nombre = bloque["text"]

        # Si hay ley_nombre, lo agregas como bloque al principio
        if ley_nombre:
            result.insert(0, {"text": ley_nombre})

        # Construye el output_data SIN ley_nombre como campo aparte
        output_data = {
            "filename": os.path.splitext(filename)[0],
            "bloques": result
        }

        output_path = os.path.join(output_folder, filename.replace(".docx", ".json"))
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(output_data, f, ensure_ascii=False, indent=2)

        print(f"¡Archivo guardado en {output_path}!")

¡Archivo guardado en C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/json_docs\Inclusión financiera.json!
¡Archivo guardado en C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/json_docs\Ley de Residuos.json!
¡Archivo guardado en C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/json_docs\Ley Defensa al Consumidor.json!
¡Archivo guardado en C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/json_docs\Licencias Especiales.json!


## IGNORAR

Lo use para debugear el preprocesamiento de información porque no estaba recuperando la información correctamente. 

In [17]:
import json

def debug_extract_texts_from_json(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    bloques = data.get("bloques", [])
    docs = []
    for i, bloque in enumerate(bloques):
        title = bloque.get("title", "").strip()
        text = bloque.get("text", "").strip()
        print(f"--- Bloque {i} ---")
        print("TITLE:", repr(title))
        print("TEXT:", repr(text))
        print("RAW BLOQUE:", bloque)
        print("-----------------")
        # Puedes devolver como string plano o como LCDocument si lo tienes importado
        if title and text:
            docs.append(f"{title}\n{text}")
        elif title:
            docs.append(title)
        elif text:
            docs.append(text)
    return docs

In [20]:
from langchain.docstore.document import Document  # O el import correcto para tu versión

def docs_as_lc_documents(docs):
    return [Document(page_content=d) for d in docs]

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def debug_chunks(documents, chunk_size=5000, chunk_overlap=500):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Total de chunks generados: {len(chunks)}")
    for i, chunk in enumerate(chunks):
        print(f"\n--- Chunk {i} ---")
        print(repr(chunk.page_content))
        print("-----------------")
    return chunks

In [26]:
def store_in_chroma(docs):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectordb = Chroma.from_documents(docs, embeddings, persist_directory=CHROMA_DB_DIR)
    vectordb.persist()
    return vectordb

def retrieve_documents(query: str):
    retriever = vectordb.as_retriever(search_kwargs={"k": 5})
    docs = retriever.invoke(query)
    if not docs:
        return None
    for i, doc in enumerate(docs):
        print(f"--- Documento {i} ---")
        print(repr(doc.page_content))
        print("---------------------")
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
import os
import json
from langchain.docstore.document import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHROMA_DB_DIR = "chroma_db"
DATA_DIR = "C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data"

docs = debug_extract_texts_from_json("C:/Users/Felipe/Documents/GitHub/modelos-lenguaje-entrega/src/data/json_docs/Ley de Residuos.json")
lc_docs = docs_as_lc_documents(docs)
chunks = debug_chunks(lc_docs, chunk_size=5000, chunk_overlap=500)
store_in_chroma(chunks)
retrieve_documents("¿Cuáles son las directivas generales para la gestión de residuos?")

--- Bloque 0 ---
TITLE: '¿Sabías que…se aprobó una Ley que regula la gestión de los residuos para proteger el medio ambiente?'
TEXT: 'Su objetivo es prevenir y reducir el impacto negativo de la generación y del manejo de los residuos y el reconocimiento de sus posibilidades de generar valor y empleo de calidad.'
RAW BLOQUE: {'title': '¿Sabías que…se aprobó una Ley que regula la gestión de los residuos para proteger el medio ambiente?', 'text': 'Su objetivo es prevenir y reducir el impacto negativo de la generación y del manejo de los residuos y el reconocimiento de sus posibilidades de generar valor y empleo de calidad.'}
-----------------
--- Bloque 1 ---
TITLE: '¿Qué es un residuo o desecho?'
TEXT: 'Toda sustancia, material u objeto del cual alguien se desprende o da disposición final.'
RAW BLOQUE: {'title': '¿Qué es un residuo o desecho?', 'text': 'Toda sustancia, material u objeto del cual alguien se desprende o da disposición final.'}
-----------------
--- Bloque 2 ---
TITLE: '¿Qu

C:\Users\Felipe\AppData\Local\Temp\ipykernel_1156\2674729225.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


NameError: name 'vectordb' is not defined